In [ ]:
!pip install sentence_transformers
!pip install sklearn
!pip install umap
!pip install hdbscan
!pip install matplotlib
!pip install plotly
!pip install scipy

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
import pandas as pd

In [ ]:
model_name ='all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

In [ ]:
def read_file(file_name):
    positives = [];
    negatives = [];

    with open(file_name, 'r') as data:
        lines = data.readlines()
        for line in lines:
            item = json.loads(line);
            sentence1 = item['text_1'].split(': ')[1]
            sentence2 = item['text_2'].split(': ')[1]
            label = item['label']
            el = {'sentence1': sentence1, sentence2: 'sentence2'}
            if label == 1: 
                positives.add({'sentence1) 
            else: 
                negatives.add(el)
    return  pd.DataFrame(positives), pd.DataFrame(positives)

positives, negatives = read_file("data_pairs.jsonl");

sentences1 = [item['sentence1'] for item in positives];
sentences2 = [item['sentence2'] for item in positives];
embeddings1_positives = model.encode(sentences1, convert_to_numpy=True, show_progress_bar=True)
embeddings2_positives = model.encode(sentences2, convert_to_numpy=True, show_progress_bar=True)
similarity_positives_matrix = cosine_similarity(embeddings1_positives, embeddings2_positives)
similarity_positives = np.diag(similarity_positives_matrix)

sentences1 = [item['sentence1'] for item in negatives];
sentences2 = [item['sentence2'] for item in negatives];
embeddings1_negatives = model.encode(sentences1, convert_to_numpy=True, show_progress_bar=True)
embeddings2_negatives = model.encode(sentences2, convert_to_numpy=True, show_progress_bar=True)
similarity_negatives_matrix = cosine_similarity(embeddings1_negatives, embeddings2_negatives)
similarity_negatives = np.diag(similarity_negatives_matrix)

t-test to check if the average similarity of sim_positive is significantly higher than that of sim_negative.

In [ ]:
# Independent t-test
t_stat, p_value = stats.ttest_ind(similarity_positives, similarity_negatives)
print("t-statistic:", t_stat)
print("p-value:", p_value)

Even if the difference is statistically significant, you also want to know how big it is. That’s what Cohen’s d measures:

In [ ]:
def cohens_d(a, b):
    mean_diff = np.mean(a) - np.mean(b)
    pooled_std = np.sqrt((np.std(a, ddof=1)**2 + np.std(b, ddof=1)**2) / 2)
    return mean_diff / pooled_std

d = cohens_d(similarity_positives, similarity_negatives)
print("Cohen's d:", d)